In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

root_path = '../'
sys.path.append(os.path.join(root_path, 'code', 'core'))

import pse 
import utils 
import numpy as np 
import itertools
import math
import networkx as nx
import matplotlib.pyplot as plt

from ranking_utils import *
from mallows import *
from ws_ranking import *
import numpy as np


[ 2  4  6  8 10 12 14 16 18]
[0.1 0.5 1. ]
(120, 120)
10 0
Reconstruction error =3.224657399706958e-16
[[0. 4. 5. ... 9. 4. 5.]
 [4. 0. 5. ... 7. 6. 7.]
 [5. 5. 0. ... 4. 3. 8.]
 ...
 [9. 7. 4. ... 0. 5. 6.]
 [4. 6. 3. ... 5. 0. 7.]
 [5. 7. 8. ... 6. 7. 0.]]
(120, 10) 10


In [ ]:
# Draw LF samples 
# 


d = 5 # num items in rankings
n = 250 
max_m = 18

np.random.seed(0)

theta_star = np.zeros(3)
theta_star = np.array([0.1,0.5,1.0])

#theta_star[:10] = np.random.uniform(0.1,0.2,10)
#theta_star[10:] = np.random.uniform(2,5,8)

lst_m = np.arange(2,19,2)
print(lst_m)
lst_out = []
trials = 5
np.sort(theta_star)
print(theta_star)
np.random.shuffle(theta_star)

def generate_true_rankings(d,n):
    r_utils = RankingUtils(d)
    Y = [r_utils.get_random_ranking() for i in range(n)]
    return Y

def sample_mallows_LFs(Y,m,thetas,p=None,d_mask=None):
    # p = None means sample full rankings
    # else each alternative will appear in at least p fraction of LFs.
    n = len(Y)
    d = len(Y[0])

    thetas = np.array(thetas)
    r_utils = RankingUtils(d)

    # get LFs
    lst_mlw = [ Mallows(r_utils,theta) for theta in thetas ]
    L = [[mlw.sample(y)[0] for mlw in lst_mlw] for y in Y]

    return L 


In [31]:
# compute psuedo-euclidean embeddings

r_utils = RankingUtils(d)
def compute_pse(Y,L, r_utils, dim=0):
    # assumes hash function is implemented by the class of obj.
    unique_obj = set(Y)
    n,m = len(L),len(L[0])
    for i in range(len(L)):
        for j in range(len(L[0])):
            unique_obj.add(L[i][j])
    
    # compute pairwise distances b/w unique objs
    unique_obj_lst = list(unique_obj)
    map_obj_id = {k: v for v, k in enumerate(unique_obj_lst)}
    map_id_obj = unique_obj_lst 

    D = [[r_utils.kendall_tau_distance(r1,r2) for r2 in unique_obj] for r1 in unique_obj]
    D = np.array(D) 
    print(D.shape)

    X, tk = pse.pseudo_embedding(D, dim)
    s = len(D)
    rec = np.zeros((s, s))
    err = 0
    for i in range(s):
        for j in range(s):
            rec[i,j] = np.abs(np.linalg.norm(X[i,:tk]-X[j,:tk])**2 - np.linalg.norm(X[i,tk:]-X[j,tk:])**2)
            if i != j:
                err += np.abs((np.sqrt(D[i,j])-np.sqrt(rec[i,j])))/np.sqrt(D[i,j])
    err /= (s*(s-1))
    print(f"Reconstruction error ={err}")
    print(D)
    Y_emb = np.array([X[map_obj_id[y]] for y in Y])
    L_emb = np.array([ [ X[map_obj_id[L[i][j]]]  for i in range(n)] for j in range(m) ] )

    return Y_emb,L_emb, X,tk 

Y = generate_true_rankings(d,n)
L = sample_mallows_LFs(Y,max_m,theta_star)

Y_emb, L_emb, X,tk = compute_pse(Y,L,r_utils,dim=0) 
print(X.shape,tk  )

#unique_obj = compute_pse(Y,L,r_utils)
#print(len(unique_obj))
#for obj in unique_obj:
#    print(obj)


(120, 120)
10 0
Reconstruction error =2.906973930800299e-16
[[0. 4. 5. ... 9. 4. 5.]
 [4. 0. 5. ... 7. 6. 7.]
 [5. 5. 0. ... 4. 3. 8.]
 ...
 [9. 7. 4. ... 0. 5. 6.]
 [4. 6. 3. ... 5. 0. 7.]
 [5. 7. 8. ... 6. 7. 0.]]
(120, 10) 10


In [ ]:
# LFs mapped to embedding space

# create tensors 




In [ ]:
# perform tensor decomposition to recover the parameters

#tensorly


In [6]:
# evaluate the recovered parameters, how ? 
# do we know true mu ? if yes, we can show distance between mu, mu_hat
# is mu the embedding of center y ? not necessary. 

# from mu_hat, mean distance and then theta_hat 

# using theta_hat draw samples 

# proceed with learning and test error 
